# Merge Dataset for each Marcroeconomic Feature for dates between 12-01-1977 to 08-01-2022

In [1]:
import os
import pandas as pd
from functools import reduce

# Path to your datasets folder
folder_path = r"C:\Users\viraj\BuildSpace\Projects\Macroeconomic-Regime-Analysis-Credit-Risk\datasets"

# List all CSV files in the folder
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# Initialize a list to hold dataframes
df_list = []

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # Read the CSV file
    df_temp = pd.read_csv(file_path)
    
    # Rename columns:
    # - The first column is the date column
    # - The second column is named after the file (minus .csv) for clarity
    base_name = os.path.splitext(file_name)[0]  # e.g., "Consumer Price Index for All Urban..."
    df_temp.columns = ["Date", base_name]
    
    # Convert Date column to datetime if not already
    df_temp["Date"] = pd.to_datetime(df_temp["Date"])
    
    # Append to list
    df_list.append(df_temp)

# Merge all dataframes on the Date column (outer join to keep all dates)
df_merged = reduce(lambda left, right: pd.merge(left, right, on="Date", how="outer"), df_list)

# Sort by Date just in case
df_merged.sort_values("Date", inplace=True)

# Save the combined dataset
output_file = os.path.join(folder_path, "macroeconomic_data_merged.csv")
df_merged.to_csv(output_file, index=False)

print(f"Merged dataset saved to: {output_file}")


Merged dataset saved to: C:\Users\viraj\BuildSpace\Projects\Macroeconomic-Regime-Analysis-Credit-Risk\datasets\macroeconomic_data_merged.csv


# Converting Credit Risk to monthly data from daily & handling missing values

In [2]:
import pandas as pd

# Define the path to your CSV file
file_path = r"C:\Users\viraj\BuildSpace\Projects\Macroeconomic-Regime-Analysis-Credit-Risk\datasets\ICE BofA US High Yield Index Option-Adjusted Spread_BAMLH0A0HYM2.csv"

# Load the data
df = pd.read_csv(file_path)

# Convert the observation_date to datetime and set as index
df['observation_date'] = pd.to_datetime(df['observation_date'])
df.set_index('observation_date', inplace=True)

# Display initial data
print("Initial Data:")
print(df.head(10))

# Handle missing values in the 'BAMLH0A0HYM2' column.
# Method 1: Forward Fill (ffill)
df_ffill = df.copy()
df_ffill['BAMLH0A0HYM2'] = df_ffill['BAMLH0A0HYM2'].ffill()

# Method 2: Interpolation
df_interp = df.copy()
df_interp['BAMLH0A0HYM2'] = df_interp['BAMLH0A0HYM2'].interpolate(method='linear')

# You can choose either method depending on your preference. Here we use forward fill.
df_clean = df_ffill

# Now, convert daily data to monthly.
# Method A: Using Monthly Mean
monthly_mean = df_clean.resample('M').mean()

# Method B: Using Last Observation of the Month
monthly_last = df_clean.resample('M').last()

# Display the resampled data for both methods
print("\nMonthly Mean Aggregation:")
print(monthly_mean.head())

print("\nMonthly Last Observation Aggregation:")
print(monthly_last.head())

# Save the monthly aggregated data back to CSV files (if needed)
monthly_mean.to_csv(r"C:\Users\viraj\BuildSpace\Projects\Macroeconomic-Regime-Analysis-Credit-Risk\datasets\credit_spread_monthly_mean.csv")
monthly_last.to_csv(r"C:\Users\viraj\BuildSpace\Projects\Macroeconomic-Regime-Analysis-Credit-Risk\datasets\credit_spread_monthly_last.csv")

print("Monthly aggregated files have been saved.")


Initial Data:
                  BAMLH0A0HYM2
observation_date              
1996-12-31                3.13
1997-01-01                 NaN
1997-01-02                3.06
1997-01-03                3.09
1997-01-06                3.10
1997-01-07                3.10
1997-01-08                3.07
1997-01-09                3.13
1997-01-10                3.16
1997-01-13                3.04

Monthly Mean Aggregation:
                  BAMLH0A0HYM2
observation_date              
1996-12-31            3.130000
1997-01-31            3.036957
1997-02-28            2.956000
1997-03-31            2.696190
1997-04-30            2.893182

Monthly Last Observation Aggregation:
                  BAMLH0A0HYM2
observation_date              
1996-12-31                3.13
1997-01-31                3.04
1997-02-28                2.73
1997-03-31                2.84
1997-04-30                2.94
Monthly aggregated files have been saved.


# Handle inconsistency in 'observation_date'

In [3]:
import pandas as pd
import os

# File paths
macro_file = r"C:\Users\viraj\BuildSpace\Projects\Macroeconomic-Regime-Analysis-Credit-Risk\datasets\macroeconomic_data_merged.csv"
credit_file = r"C:\Users\viraj\BuildSpace\Projects\Macroeconomic-Regime-Analysis-Credit-Risk\datasets\credit_spread_monthly_mean.csv"

# Load the datasets
macro_df = pd.read_csv(macro_file)
credit_df = pd.read_csv(credit_file)

# Function to standardize dates to end-of-month
def standardize_to_eom(df, date_col):
    df[date_col] = pd.to_datetime(df[date_col])
    # Convert the date to a Period (monthly) then back to timestamp representing the end of that month
    df[date_col] = df[date_col].dt.to_period('M').dt.to_timestamp('M')
    return df

# Standardize dates in both datasets
macro_df = standardize_to_eom(macro_df, 'Date')
credit_df = standardize_to_eom(credit_df, 'observation_date')

# Merge the datasets on the standardized date columns
merged_df = pd.merge(macro_df, credit_df, left_on='Date', right_on='observation_date', how='outer')

# Optionally, drop duplicate date columns or rename for clarity
merged_df.drop(columns=['observation_date'], inplace=True)
merged_df.rename(columns={'Date': 'Month_End'}, inplace=True)

# Sort by the date
merged_df.sort_values('Month_End', inplace=True)

# Save the merged dataset
output_path = os.path.join(os.path.dirname(macro_file), "merged_macroeconomic_credit.csv")
merged_df.to_csv(output_path, index=False)

print(f"Merged dataset saved to: {output_path}")


Merged dataset saved to: C:\Users\viraj\BuildSpace\Projects\Macroeconomic-Regime-Analysis-Credit-Risk\datasets\merged_macroeconomic_credit.csv
